In [1]:
import pandas as pd
import os
from modules.Date2 import find_starting_point, find_specific_date, convert_date
from modules.variables2 import *
from modules.Data_Quality import *
from modules.Cost_centre_structure import *

In [21]:
end_month = 'Apr'
end_year = '2023/24'

files_info =[]
dataframes = []

sheet_name = 'Leeds'
root = './'
input_folder = root+'Input files/' + 'forecast_12_May_2023_input'
output_folder = root+'Prepared Files/' + 'Forecasts_12_Month_May_23'
combined_folder = root+'Combined files/' +'Forecasts_12_Month_May_23_Combined/'
directory = os.fsencode(input_folder)
num = 0

In [10]:
def process_file(file_path, division):
    global num
    xls = pd.ExcelFile(file_path)
    subregion_dict = division_to_subregion.get(division)

    if not subregion_dict:
        print(f"No subregion dictionary found for division {division}")
        return

    for sheet_name in xls.sheet_names:
        if any(sheet_name in cost_centres for cost_centres in subregion_dict.values()):
            num += 1
            # Load the data from the sheet, specifying no header to properly identify the row
            data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
            print(division, sheet_name)
            start_month, start_year, row, start_col = find_starting_point(data)
            end_row, end_col = find_specific_date(data, end_month, end_year)
            dates = convert_date(data, row, start_col, end_col)

            extracted_data = extract_var_columns(data, start_col, end_col)
            extracted_data, region, division, cost_centre = get_metadata(extracted_data, division, sheet_name)
            final_df = pd.DataFrame({'Date': dates})
            final_df = pd.concat([final_df, extracted_data.reset_index(drop=True)], axis=1)
            num_col = len(final_df.columns)
            has_missing, rows_with_missing = check_missing_values(final_df, exclude_column)
            has_secured_missing = check_missing_secured(final_df)
            duplicated = check_duplicates(final_df)
            incorrect_relationship = check_relationships(final_df)

            var_name = f"{os.path.basename(file_path)}_{sheet_name}_missing"
            globals()[var_name] = rows_with_missing

            file_info = {'file_name': os.path.basename(file_path).split('.')[0],
                         'num': num,
                         'region': region,
                         'division': division,
                         'cost_centre': cost_centre,
                         'num_col': num_col,
                         'Start month': start_month,
                         'Start year': start_year,
                         'row': row,
                         'End column': end_col,
                         'missing': has_missing,
                         'secured missing': has_secured_missing,
                         'duplicated': duplicated,
                         'incorrect_relationship': incorrect_relationship}

            files_info.append(file_info)
            dataframes.append(final_df)
            output_file = f"{division}_{sheet_name.replace(' ', '_')}"
            prepared_file_path = os.path.join(output_folder, f'{output_file}_ok.csv')
            final_df.to_csv(prepared_file_path, index=False)

In [11]:
# Main loop to process all files
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") or filename.endswith(".xls") or filename.endswith(".xlsm"):
        file_path = os.path.join(input_folder, filename)

        # Extract division from filename
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        name_parts = base_name.split('-')
        if len(name_parts) > 1:
            division = name_parts[1].strip().lower()
            process_file(file_path, division)

files_info_df = pd.DataFrame(files_info)
files_info_df.to_csv('files_info.csv', index=False)

advisory AFM
advisory Pro
advisory R&H
advisory NZ
advisory Proc
advisory Disp
advisory SHQUK
advisory SHQTel
advisory Suiko
advisory AdvCo
amcl Group
amcl UK&I
amcl AMA
amcl Vertex
amcl IES
cost management Edinburgh
cost management Glasgow
cost management Belfast
cost management NorthEast
cost management Leeds
cost management NorthWest
cost management Sheffield
cost management Birmingham
cost management Nottingham
cost management Bristol
cost management Cambridge
cost management CLGov
cost management HiTech
cost management HE
cost management Occupier
cost management Developer
cost management alinea
cost management Co
infrastructure CM Scotland
infrastructure PM Scotland
infrastructure PMO Scotland
infrastructure CM NW
infrastructure PM NW
infrastructure PMO NW
infrastructure Def Nth
infrastructure CM YNE
infrastructure PM YNE
infrastructure PMO YNE
infrastructure CM SE
infrastructure PM SE
infrastructure PMO SE
infrastructure Def SE
infrastructure T&U SW
infrastructure Def CM SW
infra

In [12]:
len(dataframes)

76

In [13]:
excluded_divisions = ['amcl', 'advisory']  # Add the names of divisions to exclude
excluded_cost_centres = ['PM_RE_Digital', 'CM_alinea', 'INF_Def_Nth', 'INF_Def_SE','INF_T&U_SW',
                         'INF_Def_CM_SW','INF_Def_P3M_SW', 'INF_Digital', 'PM_Pcon_Sth', 'PM_Co']  # Add the names of cost centres to exclude

# Filter dataframes to exclude problematic divisions and cost centres
filtered_dataframes = []
for df in dataframes:
    if df['Division'].iloc[0].lower() not in excluded_divisions and df['Cost_Centre'].iloc[0] not in excluded_cost_centres:
        filtered_dataframes.append(df)

In [14]:
len(filtered_dataframes)

51

In [15]:
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

In [16]:
# Define columns to keep and columns to rename
# Calculate `income_ex_biso` if it's not already calculated
if 'income_ex_biso' not in combined_df.columns:
    combined_df['income_ex_biso'] = combined_df[['secured', 'probable', 'possible', 'speculative']].sum(axis=1)
    
columns_to_keep = [
    'Date','income_ex_biso', 'budget_income', 'direct_costs', 'overheads',
    'net_profit', 'budget_profit', 'total_headcount',
    'total_income', 'Region', 'Cost_Centre', 'Division' 
]

combined_df = combined_df[columns_to_keep]

cols_not_rename = ['Region', 'Cost_Centre', 'Division', 'Date']
# Create a list of columns to rename (exclude columns_to_keep and 'Region', 'Cost_Centre', 'Division', 'Date')
columns_to_rename = [col for col in combined_df.columns if col not in cols_not_rename]

# Dictionary to map old names to new names
rename_mapping = {col: col + '_Fcst12' for col in columns_to_rename}

# Rename columns in the dataframe
combined_df_fcst = combined_df.rename(columns=rename_mapping)

In [19]:
combined_df.head()

,Date,income_ex_biso_Fcst12,budget_income_Fcst12,direct_costs_Fcst12,overheads_Fcst12,net_profit_Fcst12,budget_profit_Fcst12,total_headcount_Fcst12,total_income_Fcst12,Region,Cost_Centre,Division
0,2020-05-01,64.49199,81,33.01441,13.29881,18.17877,32.995676,7.12,64.49199,other_regions,CM_Edinburgh,Cost management
1,2020-06-01,71.80276,66,42.95780,13.49737,21.52159,17.709312,7.09,77.97676,other_regions,CM_Edinburgh,Cost management
2,2020-07-01,67.24102,62,47.51482,13.20457,16.42747,13.328426,7.09,77.14686,other_regions,CM_Edinburgh,Cost management
3,2020-08-01,75.23662,66.15,53.02678,17.37175,11.19208,3.188614,7.09,81.59061,other_regions,CM_Edinburgh,Cost management
4,2020-09-01,57.85373,66.51,44.17022,18.73223,2.77168,3.535208,7.09,65.67413,other_regions,CM_Edinburgh,Cost management


In [26]:
# Read the CSV file
Folder = root + 'Combined files/'
combined_file_path = Folder + 'combined_file_1.csv'
combined_file_df = pd.read_csv(combined_file_path)

# Ensure date columns are in datetime format
combined_file_df['Date'] = pd.to_datetime(combined_file_df['Date'])
combined_df_fcst['Date'] = pd.to_datetime(combined_df_fcst['Date'])

# Define the date range
start_date = '2023-05-01'
end_date = '2024-04-01'

# Filter combined_df_fcst for the specified date range
combined_df_fcst_filtered = combined_df_fcst[
    (combined_df_fcst['Date'] >= start_date) & (combined_df_fcst['Date'] <= end_date)
    ]

# List of columns to add
columns_to_add = [
    'income_ex_biso_Fcst12', 'budget_income_Fcst12', 'direct_costs_Fcst12',
    'overheads_Fcst12', 'net_profit_Fcst12', 'budget_profit_Fcst12',
    'total_headcount_Fcst12', 'total_income_Fcst12'
]

# Add new columns to combined_file_df with NaN values initially
for column in columns_to_add:
    combined_file_df[column] = pd.NA

# Fill in the new columns with values from combined_df_fcst_filtered
for index, row in combined_file_df.iterrows():
    matching_rows = combined_df_fcst_filtered[
        (combined_df_fcst_filtered['Cost_Centre'] == row['Cost_Centre']) &
        (combined_df_fcst_filtered['Date'] == row['Date'])
        ]
    if not matching_rows.empty:
        for column in columns_to_add:
            combined_file_df.at[index, column] = matching_rows.iloc[0][column]

# Display the merged dataframe
print(combined_file_df)

      Unnamed: 0       Date    secured  probable  possible  speculative  biso  \
0              0 2020-05-01   64.49199       0.0       0.0          0.0   NaN   
1              1 2020-06-01   71.80276       0.0       0.0          0.0   NaN   
2              2 2020-07-01   67.24102       0.0       0.0          0.0   NaN   
3              3 2020-08-01   75.23662       0.0       0.0          0.0   NaN   
4              4 2020-09-01   57.85373       0.0       0.0          0.0   NaN   
...          ...        ...        ...       ...       ...          ...   ...   
2587        2587 2023-12-01  621.60097       0.0       0.0          0.0   NaN   
2588        2588 2024-01-01  577.61932       0.0       0.0          0.0   NaN   
2589        2589 2024-02-01  661.82891       0.0       0.0          0.0   NaN   
2590        2590 2024-03-01  603.66999       0.0       0.0          0.0   NaN   
2591        2591 2024-04-01  597.00000       0.0       0.0          0.0   NaN   

      budget_income  other_

In [29]:
# Save the combined dataframe to a CSV file
combined_csv_path = combined_folder + 'combined_file_fcst12.csv'
combined_file_df.to_csv(combined_csv_path, index=True)

excluded_dataframes = [df for df in dataframes if df['Division'].iloc[0] in excluded_divisions or df['Cost_Centre'].iloc[0] in excluded_cost_centres]
for df in excluded_dataframes:
    print(f"Excluded Division: {df['Division'].iloc[0]}, Cost Centre: {df['Cost_Centre'].iloc[0]}")